In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
def data_generator(r, l, e, tam=1000, rho=False):
    '''
        Entrada:
            r, l, e: Numeros Naturais maiores que zero
            tam : Valor maximo de cada elemento, padrao: 1000.
            rho : Flag para retornar apenas uma matriz l x r
        Saida:
            vetor: tamanho r, matriz: ordem l x r, matriz: ordem e x r
        Erro:
            tupla: None, print do erro.
    '''
    less_than = l <= (r - 1) and e <= (r - 1)
    greather_than_zero = l > 0 and e > 0 and r > 0

    if not (less_than and greather_than_zero):
        print("Data Generator Error: Bad Entry.")
        return (None, None, None)
    else:
        if rho:
            matriz_lr = np.random.randint(0, tam + 1, size=(l, r))
            return (matriz_lr)
        else:
            vector = np.random.randint(0, tam + 1, size=r)
            matriz_lr = np.random.randint(0, tam + 1, size=(l, r))
            matriz_er = np.random.randint(0, tam + 1, size=(e, r))

        return (vector, matriz_lr, matriz_er)

# ---------------------------------------------------------------------------------------------------

def generator_conditional(r, l, e, tam=1000):
    """
        Entradas:
            r, l, e: Numeros naturais maiores que zero
            tam = tamanho maximo para numero aleatorio
            t = variação maxima do gerador do conjunto Q
        Saidas:
            c: Vetor de tamanho r
            rho: Matriz de ordem l x r
            pi: Matriz de ordem e x r
            rho_r: vetor de tamanho l
            pi_r: vetor de tamanho e
            Q: Conjunto de tamanho r + 1 tal que U0 = Ur
    """

    c, rho, pi = data_generator(r, l, e, tam=tam)

    # Ultima coluna das matrizes Rho e Pi respectivamente
    rho_r = rho[:, -1:]
    pi_r = pi[:, -1:]

    return (c, rho, pi, rho_r, pi_r)

In [3]:
def to_list(vet):
    lista = []
    for x in vet:
        lista.append(x[0])

    return np.array(lista)

In [4]:
def cria_tabela(r, c, rho, pi, rho_r, pi_r):
    ident = np.eye(r)
    zeros = np.zeros(r)
    
    # Concatena as função objetivo
    # Concatena as restriçoes
    # Adiciona as variaveis de folga
    # Adiciona zeros a funcao objetivo
    # Multiplica funcao objetivo por -1 para ficar todos negativos
    # Adiciona um zero ao c
    # Modifica o shape do c para (r+1, 1)
    funcao_objetivo = np.concatenate((rho_r, pi_r))
    restricao = np.concatenate((rho.T, pi.T), axis=1)
    funcao_objetivo = to_list(funcao_objetivo)
    restricao_expandida = np.concatenate((restricao, ident), axis = 1)
    funcao_objetivo = np.concatenate((funcao_objetivo, zeros))
    funcao_objetivo = np.dot(-1, funcao_objetivo)
    c = np.concatenate(([0], c))
    c = np.array([c]).T
    
    # Concatenação Geral
    horizontal = np.concatenate(([funcao_objetivo], restricao_expandida))
    vertical = np.concatenate((horizontal, c), axis=1)
    
    # Retorna uma tabela
    return vertical

In [5]:
def filtro(tabela):
    # Pega o minimo da primeira linha
    menor = np.min(tabela[0])
    # Indice do minimo da primeira linha
    indice = np.where(tabela[0] == menor)[0][0]
    # Pega a coluna pertencente ao minimo
    colum = tabela[:, indice:indice+1]
    # Pega a ultima coluna da tabela
    ult_colum = tabela[:, -1:]
    
    # Iteraçao para realizar as divisões da ultima coluna pela coluna de indice do menor
    divisao = []
    n = colum.size
    i = 0
    while i < n:
        divisao.append(ult_colum[i]/colum[i])
        i+=1
    
    # Minimo valor das divisoes
    minimo = np.min(divisao)
    
    # Divisão do valor i da ultima coluna pelo i da coluna do indice menor
    valor_de_verificacao = ult_colum[-1]/colum[-1]
    
    if minimo == valor_de_verificacao:
        return True
    else:
        return False

In [122]:
def aplica_filtro(r):
    c, rho, pi, rho_r, pi_r = generator_conditional(r, r-1, r-1)
    tabela = cria_tabela(r, c, rho, pi, rho_r, pi_r)
    verifica = filtro(tabela)
    if verifica:
        (c, rho, pi, rho_r, pi_r) = aplica_filtro(r)
    
    return (c, rho, pi, rho_r, pi_r)

In [123]:
def prepara_dual(c, rho, pi, rho_r, pi_r):
    funcao_objetivo = np.concatenate((rho_r, pi_r))
    restricao = np.concatenate((rho.T, pi.T), axis=1)
    funcao_objetivo = to_list(funcao_objetivo)
    funcao_objetivo = np.dot(-1, funcao_objetivo)
    
    return(funcao_objetivo, restricao, c)

In [124]:
c, rho, pi, rho_r, pi_r = aplica_filtro(3)
objetivo, restricao, vetor = prepara_dual(c, rho, pi, rho_r, pi_r)

In [125]:
dual = linprog(c=objetivo, A_ub=restricao, b_ub=vetor)
dual

     fun: -261.0
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([ 54.65343348, 204.84120172,   0.        ])
  status: 0
 success: True
       x: array([0.        , 0.28004292, 0.        , 0.        ])

In [126]:
primal = linprog(c=c, A_ub=rho, b_ub=rho_r, A_eq=pi, b_eq=pi_r)
primal

     fun: 261.0
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([0., 0.])
  status: 0
 success: True
       x: array([0., 0., 1.])